In [1]:
import dask
import dask.dataframe as dd

In [2]:
# InterProScan default column names
COLUMNS = ['prot_access', 'sequ_md5_dig', 'seq_len',
'analysis', 'sign_access', 'sign_descr', 'start_loc', 'stop_loc',
'score', 'status', 'date', 'iPro_access', 'iPro_descr',
'GO_annot', 'pathway']


In [29]:
# test data
# path = 'data/mock_data.csv'
# path = 'data/mock_data_100000.csv'

path='/data/datsaprocessing/interproscan/all_bacilli.tsv'

# df = dd.read_csv(path, dtype={'score':'object'}, blocksize='500MB').set_index('prot_access')
df = dd.read_csv(path, sep='\t', names=COLUMNS, dtype={'score': 'object'}, blocksize='500MB').set_index('prot_access')

In [4]:
df.head(1)

,sequ_md5_dig,seq_len,analysis,sign_access,sign_descr,start_loc,stop_loc,score,status,date,iPro_access,iPro_descr,GO_annot,pathway
prot_access,,,,,,,,,,,,,,
gi|269850218|gb|AAP25868.2|,1a2e1b20af7796eaa0970aed49034347,465,CDD,cd00075,HATPase,364,463,4.4491E-31,T,25-04-2022,-,-,NaN,NaN


In [5]:
df.tail(1)

,sequ_md5_dig,seq_len,analysis,sign_access,sign_descr,start_loc,stop_loc,score,status,date,iPro_access,iPro_descr,GO_annot,pathway
prot_access,,,,,,,,,,,,,,
gi|30260184|gb|AAP29369.1|,497e9c4a1564a7769e7dd6a69b2e8e50,44,MobiDBLite,mobidb-lite,consensus disorder prediction,30,44,-,T,25-04-2022,-,-,NaN,NaN


In [6]:
selection = df[df['iPro_access'] != '-']
selection = selection[['seq_len', 'iPro_access', 'start_loc', 'stop_loc']]

In [7]:
selection.head()

,seq_len,iPro_access,start_loc,stop_loc
prot_access,,,,
gi|269850218|gb|AAP25868.2|,465,IPR003661,248,314
gi|269850218|gb|AAP25868.2|,465,IPR005467,255,465
gi|269850218|gb|AAP25868.2|,465,IPR003594,359,465
gi|269850218|gb|AAP25868.2|,465,IPR003660,192,243
gi|269850218|gb|AAP25868.2|,465,IPR036097,231,316


In [8]:
selection.tail(1)

,seq_len,iPro_access,start_loc,stop_loc
prot_access,,,,
gi|30260184|gb|AAP29369.1|,44,IPR020939,2,21


In [9]:
selection['difference'] = (selection['stop_loc'].astype(float) - selection['start_loc'].astype(float)) / selection['seq_len'].astype(float)

In [10]:
selection.head()

,seq_len,iPro_access,start_loc,stop_loc,difference
prot_access,,,,,
gi|29893821|gb|AAP07113.1|,70,IPR036986,1,70,0.985714
gi|29893821|gb|AAP07113.1|,70,IPR014330,4,62,0.828571
gi|29893849|gb|AAP07141.1|,660,IPR041872,391,531,0.212121
gi|29893849|gb|AAP07141.1|,660,IPR002547,566,658,0.139394
gi|29893849|gb|AAP07141.1|,660,IPR014729,8,346,0.512121


In [10]:
computed = selection[['iPro_access', 'difference']]
# computed.head(1)

In [11]:
test = computed.reset_index()
test = test.categorize(columns=['iPro_access'])

In [12]:
piv = dask.dataframe.reshape.pivot_table(test, index='prot_access', columns='iPro_access')

In [13]:
piv.head()

difference                                \
iPro_access                  IPR003661 IPR005467 IPR003594 IPR003660   
prot_access                                                            
gi|269850218|gb|AAP25868.2|   0.141935  0.451613  0.227957  0.111111   
gi|269850219|gb|AAP29310.2|        NaN       NaN       NaN       NaN   
gi|269850220|gb|AAP29073.2|        NaN       NaN       NaN       NaN   
gi|269850221|gb|AAP24130.2|        NaN       NaN       NaN       NaN   
gi|269850222|gb|AAP28537.2|        NaN       NaN       NaN       NaN   

                                                                               \
iPro_access                 IPR036097 IPR004358 IPR036890 IPR029044 IPR001173   
prot_access                                                                     
gi|269850218|gb|AAP25868.2|  0.182796  0.032258  0.335484       NaN       NaN   
gi|269850219|gb|AAP29310.2|       NaN       NaN       NaN  0.809524  0.424908   
gi|269850220|gb|AAP29073.2|       NaN       NaN       NaN       NaN       NaN   
gi|269850221|gb|AAP24130.2|       NaN       NaN       NaN       NaN       NaN   
gi|269850222|gb|AAP28537.2|       NaN       NaN       NaN       NaN       NaN   

                                       ...                                \
iPro_access                 IPR005286  ... IPR022896 IPR023599 IPR015860   
prot_access                            ...                                 
gi|269850218|gb|AAP25868.2|       NaN  ...       NaN       NaN       NaN   
gi|269850219|gb|AAP29310.2|       NaN  ...       NaN       NaN       NaN   
gi|269850220|gb|AAP29073.2|  0.929825  ...       NaN       NaN       NaN   
gi|269850221|gb|AAP24130.2|       NaN  ...       NaN       NaN       NaN   
gi|269850222|gb|AAP28537.2|       NaN  ...       NaN       NaN       NaN   

                                                                               \
iPro_access                 IPR008929 IPR031680 IPR012480 IPR011123 IPR025099   
prot_access                                                                     
gi|269850218|gb|AAP25868.2|       NaN       NaN       NaN       NaN       NaN   
gi|269850219|gb|AAP29310.2|       NaN       NaN       NaN       NaN       NaN   
gi|269850220|gb|AAP29073.2|       NaN       NaN       NaN       NaN       NaN   
gi|269850221|gb|AAP24130.2|       NaN       NaN       NaN       NaN       NaN   
gi|269850222|gb|AAP28537.2|       NaN       NaN       NaN       NaN       NaN   

                                                 
iPro_access                 IPR025152 IPR007267  
prot_access                                      
gi|269850218|gb|AAP25868.2|       NaN       NaN  
gi|269850219|gb|AAP29310.2|       NaN       NaN  
gi|269850220|gb|AAP29073.2|       NaN       NaN  
gi|269850221|gb|AAP24130.2|       NaN       NaN  
gi|269850222|gb|AAP28537.2|       NaN       NaN  

[5 rows x 5673 columns]

In [14]:
y = piv.idxmax(axis=1)
t = y.compute()
_, labels = t.str

/tmp/ipykernel_2233081/2322953700.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  _, labels = t.str


In [15]:
X = piv.notnull().astype(int)
# X.head()

# machine learning part

In [16]:
from dask_ml.model_selection import train_test_split
from dask_ml.wrappers import Incremental
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
import dask.array as da

In [17]:
enc = LabelEncoder()
enc.fit(labels)

LabelEncoder()

In [18]:
y_labels = enc.transform(labels)

In [19]:
X_arr = X.to_dask_array(lengths=True)
y_arr = da.from_array(y_labels, chunks=X_arr.chunksize[0])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size=0.2, random_state=0)

### test

In [21]:
# test with wrapper
from dask.distributed import Client
import joblib

client = Client(processes=False)

/commons/conda/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35721 instead
  warnings.warn(


In [22]:
from sklearn.ensemble import RandomForestClassifier

try:
	model = RandomForestClassifier(random_state=0)
	with joblib.parallel_backend('dask'):
		model.fit(X_train, y_train)
except Exception as e:
	print(e)

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://192.168.11.130/2233081/1 remote=inproc://192.168.11.130/2233081/10>


In [23]:
model

RandomForestClassifier(random_state=0)

In [24]:
model.score(X_test, y_test)

0.825062034739454

In [25]:
from sklearn.ensemble import AdaBoostClassifier

In [26]:
clf = AdaBoostClassifier(random_state=0, n_estimators=100)
with joblib.parallel_backend('dask'):
	clf.fit(X_train, y_train)

clf.score(X_test, y_test)

3.0397022332506202e-02

In [27]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)

with joblib.parallel_backend('dask'):
	clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.6569478908188585

In [1]:
import pandas as pd

In [3]:
data = {'model': ['random forest', 'logistic regression'], 'mean accuracy': [0.825062034739454, 0.6569478908188585]}
sc = pd.DataFrame(data).set_index('model')
sc

,mean accuracy
model,
random forest,0.825062
logistic regression,0.656948
